# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file, encoding="UTF-8")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,22.80,94,100,2.57,US,1693126262
1,1,port-aux-francais,-49.3500,70.2167,6.61,96,100,21.01,TF,1693126263
2,2,adamstown,-25.0660,-130.1015,19.58,61,54,7.00,PN,1693126263
3,3,port alfred,-33.5906,26.8910,18.43,52,67,2.24,ZA,1693126054
4,4,puerto baquerizo moreno,-0.9000,-89.6000,23.38,79,81,5.61,EC,1693126264


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

coordinates = {
    "latitude": city_data_df['Lat'],
    "longitude": city_data_df['Lng'],
    "City": city_data_df['City'],
    "Humidity": city_data_df['Humidity']
}

coordinates_df = pd.DataFrame(coordinates)

#Configure the map plot
city_map_humidity = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM", 
    frame_width = 800,
    frame_height = 500, 
    size = "Humidity", 
    scale = 0.7,
    alpha = 0.7,
    color = "City",
    
)

# Display the map plot
city_map_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# My ideal temp Max temp <25 & Humidity <40%
my_weather_df = city_data_df.loc[(city_data_df['Max Temp'] < 25) & (city_data_df['Humidity'] < 40)].dropna()

# Display sample data
my_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,rikaze,29.2500,88.8833,21.73,30,71,1.08,CN,1693126271
65,65,yeniseysk,58.4497,92.1703,21.58,39,23,2.51,RU,1693126297
66,66,volksrust,-27.3654,29.8818,13.07,33,1,3.51,ZA,1693126263
97,97,sao vicente,-23.9631,-46.3919,16.26,26,100,0.45,BR,1693126313
111,111,dzhangala,49.2167,50.3333,23.71,26,16,6.18,KZ,1693126321


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_weather_df[['City_ID','City','Country','Lat','Lng','Humidity']].set_index("City_ID", drop=True)
hotel_df.index.rename(None, inplace=True)

# # Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# # Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
15,rikaze,CN,29.2500,88.8833,30,
65,yeniseysk,RU,58.4497,92.1703,39,
66,volksrust,ZA,-27.3654,29.8818,33,
97,sao vicente,BR,-23.9631,-46.3919,26,
111,dzhangala,KZ,49.2167,50.3333,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = (f"circle:{longitude},{latitude},{radius}")
    params["bias"] = (f"proximity:{longitude},{latitude}")
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

print ("-----------------------------------------------------------------------------------------")
# Display sample data
hotel_df

Starting hotel search
rikaze - nearest hotel: Gesar Hotel
yeniseysk - nearest hotel: Отель Енисейский
volksrust - nearest hotel: No hotel found
sao vicente - nearest hotel: Pousada Vitória
dzhangala - nearest hotel: Общежитие Жангалинского ЛПУ
qusmuryn - nearest hotel: No hotel found
phuthaditjhaba - nearest hotel: No hotel found
molo - nearest hotel: No hotel found
vikulovo - nearest hotel: OOO "СИСТЕМА СТРОЙ"
broken hill - nearest hotel: Ibis Styles
beyneu - nearest hotel: Johan yu
mandalgovi - nearest hotel: АЛТАНГОВЬ Hotel
murghob - nearest hotel: Pamir Hotel
alice springs - nearest hotel: Aurora Alice Springs
calama - nearest hotel: Apart Hotel Geotel Calama
kargil - nearest hotel: Border view
chernyshevskiy - nearest hotel: No hotel found
mineros - nearest hotel: No hotel found
borovskoy - nearest hotel: Маржан
zastron - nearest hotel: No hotel found
colchani - nearest hotel: Hotel de Sal - Sol de Mañana
xilin hot - nearest hotel: Hotel hailibu 120
esil - nearest hotel: Рахат
eme

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map plot
hotel_df_map 